In [18]:
import os
import io
import librosa
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from keras import layers
from keras import models
from keras import Input
from sklearn.model_selection import train_test_split

In [19]:
# Paths to data
gunshot_path = "../data/preprocessed/gunshot"
non_gunshot_path = "../data/preprocessed/not-gunshot"

# Verify that paths exists
print(os.path.exists(gunshot_path))
print(os.path.exists(non_gunshot_path))

True
True


In [27]:
model = models.Sequential()

# -> (256, 256, 3)
model.add(Input(shape=(256, 256, 3)))

# (256, 256, 3) -> (254, 254, 32)
model.add(layers.Conv2D(32, (3, 3), activation='relu'))

# (254, 254, 32) -> (127, 127, 32)
model.add(layers.MaxPooling2D((2, 2)))

# (127, 127, 32) -> (125, 125, 64)
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# (125, 125, 64) -> (62, 62, 64)
model.add(layers.MaxPooling2D((2, 2)))

# (62, 62, 64) -> (60, 60, 128)
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

# (30, 30, 128)
model.add(layers.MaxPooling2D((2, 2)))

# (30, 30, 128) -> 115,200
model.add(layers.Flatten())

# 115,200 -> 512
model.add(layers.Dense(512, activation='relu'))

# 512 -> 256
model.add(layers.Dense(256, activation='relu'))

# 256 -> 1
model.add(layers.Dense(1, activation='sigmoid'))


In [28]:
# Compile the model:
# Optimizer -> adam
# loss function -> binary_crossentropy
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
# Save model
model_name = "test_model"
model.save(f'../models/{model_name}.h5')